In [153]:
from flask import jsonify
from connect_db import execute_query
import pandas as pd
import pickle
import numpy as np
from app import app  # Importer l'application Flask

In [154]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [155]:
# clean_olympic_results
# id	discipline_title	event_title	slug_game	participant_type	medal_type	athletes	rank_equal	rank_position	country_name	country_code	country_3_letter_code
def load_data_from_database():
    with app.app_context():
        query = "SELECT * FROM clean_olympic_results"
        results_data = execute_query(query)
        results_df = pd.DataFrame(results_data)
    
    return results_df

# Charger les données
results_df = load_data_from_database()

Connexion réussie
Connexion fermée


In [156]:
# Prediction pour le top 5 des pays pour chaque discipline
results_df = results_df[results_df['rank_position'] <= 5]

# Supprimer la colonne 'athletes'
if 'athletes' in results_df.columns:
    results_df.drop(columns=['athletes'], inplace=True)
    # Supprimer la colonne 'id'
    results_df.drop(columns=['id'], inplace=True)

# supprimer une valeur rugby
results_df = results_df[results_df['discipline_title'] != 'Rugby']

# Supprimer les lignes avec des valeurs NaN
results_df.dropna(inplace=True)

# Encoder les variables catégorielles
encoder = LabelEncoder()
categorical_columns = ['discipline_title', 'event_title', 'participant_type', 'medal_type', 'country_name', 'country_code', 'country_3_letter_code', 'slug_game', 'rank_equal', 'rank_position']
for column in categorical_columns:
    results_df[column] = encoder.fit_transform(results_df[column])
    
# Séparer les données en variables explicatives et variable cible
X = results_df.drop(columns=['medal_type'])
y = results_df['medal_type']

# Séparer les données en données d'entraînement et données de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer le modèle
model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
model.fit(X_train, y_train)

# Prédire les données de test
y_pred = model.predict(X_test)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9973768928103016


In [128]:
print(results_df['discipline_title'].unique())

[52 23 14 21 31 11  3 35 50 19 64 56 59 13 42 63 66  8 60 22 29  5 68 55
 70 61  2 32 18 39 53  1 69  7 24 43  4 57 51 20 48 40  0 46 36 10 17 49
 34 28 33 58 62 27 26  6 54 12 44 37 65 25 30 38 67 41 45 47 15 16  9]


In [152]:
judo_results_df = results_df[results_df['discipline_title'] == 'Judo']

# Count occurrences and sort by count
country_counts = judo_results_df.groupby('country_name')['country_name'].count().sort_values(ascending=False)

# Select top 5 countries
top_5_judo_countries = country_counts.head(5)

# Print the top 5 countries
print("Top 5 countries in Judo:")
print(top_5_judo_countries)

Top 5 countries in Judo:
Series([], Name: country_name, dtype: int64)
